In [1]:
# Pkg.checkout("LowRankModels")
using LowRankModels, Plots

LowRankModels.jl is a julia package for modeling and fitting generalized low rank models (GLRMs). GLRMs model a data array by a low rank matrix, and include many well known models in data analysis, such as principal components analysis (PCA), matrix completion, robust PCA, nonnegative matrix factorization, k-means, and many more.

LowRankModels.jl makes it easy to mix and match loss functions and regularizers to construct a model suitable for a particular data set. In particular, it supports

   * using different loss functions for different columns of the data array, which is useful when data types are heterogeneous (eg, real, boolean, and ordinal columns);
   * fitting the model to only some of the entries in the table, which is useful for data tables with many missing (unobserved) entries; and
   * adding offsets and scalings to the model without destroying sparsity, which is useful when the data is poorly scaled.


# Losses

You've already seen most of the loss function available in LowRankModels:

   * quadratic loss - QuadLoss()
   * hinge loss - HingeLoss()
   * logistic loss - LogisticLoss()
   * poisson loss - PoissonLoss()
   * weighted hinge loss - WeightedHingeLoss()
   * l1 loss - L1Loss()
   * ordinal hinge loss - OrdinalHingeLoss()
   * periodic loss - PeriodicLoss()
   * multinomial categorical loss - MultinomialLoss()
   * multinomial ordinal (aka ordered logit) loss - OrderedMultinomialLoss()

In [2]:
# example loss function
loss = QuadLoss()

LowRankModels.QuadLoss(1.0, LowRankModels.RealDomain())

# Regularizer

We've also seen many of the regularizers available:

   * quadratic regularization - QuadReg()
   * constrained squared euclidean norm - QuadConstraint()
   * l1 regularization - OneReg()
   * no regularization - ZeroReg()
   * nonnegative constraint - NonNegConstraint() (eg, for nonnegative matrix factorization)
   * 1-sparse constraint - OneSparseConstraint() (eg, for orthogonal NNMF)
   * unit 1-sparse constraint - UnitOneSparseConstraint() (eg, for k-means)
   * simplex constraint - SimplexConstraint()
   * l1 regularization, combined with nonnegative constraint - NonNegOneReg()
   * fix features at values y0 - FixedLatentFeaturesConstraint(y0)

In [5]:
# regularizers
lambda = 1

nonneg = NonNegConstraint()
l1 = OneReg(lambda)
l2 = QuadReg(lambda)

LowRankModels.QuadReg(1.0)

# Generalized Low Rank Models

GLRMs form a low rank model for tabular data A with m rows and n columns, which can be input as an array or any array-like object (for example, a data frame). It is fine if only some of the entries have been observed (i.e., the others are missing or NA); the GLRM will only be fit on the observed entries $\Omega$.

The desired model is specified by choosing a rank k for the model, an array of loss functions losses, and two regularizers, $r_x$ and $r_w$. The data is modeled as $X^TW$, where $X$ is a $k\times m$ matrix and $W$ is a $k\times n$ matrix. $X$ and $W$ are found by solving the optimization problem

$$\min \sum_{(i,j) \in \Omega} \ell_j\bigg((X^TW)[i,j], Y[i,j]\bigg) + \sum_i r_x(X[:,i]) + \sum_j r_y(W[:,j])$$

To form a GLRM, the user specifies

   * the data $Y$ (any AbstractArray, such as an array, a sparse matrix, or a data frame)
   * the array of loss functions $\ell$
   * the regularizers $r_x$ and $r_w$
   * the rank $k$
   * the observations $\Omega$


In [16]:
# example
Y = randn(10, 10)
loss = QuadLoss()
nonneg = NonNegConstraint()
k = 5
Ω = [(rand(1:10), rand(1:10)) for iobs in 1:50] # observe 50 random entries, with replacement
glrm = GLRM(Y, loss, nonneg, nonneg, k, obs=Ω);

In [17]:
# To fit the model, call
X,W,ch = fit!(glrm);

Fitting GLRM
Iteration 10: objective value = 42.79839807412798
Iteration 20: objective value = 42.44305181292925
Iteration 30: objective value = 41.84121984741759
Iteration 40: objective value = 41.08316075366379
Iteration 50: objective value = 40.08735510405551
Iteration 60: objective value = 39.4364160191657
Iteration 70: objective value = 39.13681059192923
Iteration 80: objective value = 38.51283316709613
Iteration 90: objective value = 37.56213590292802
Iteration 100: objective value = 37.145576477529


([0.0 0.0 … 0.0 0.0; 0.0 0.141492 … 0.0 0.0; … ; 0.0 0.156508 … 0.0 0.0; 0.0 0.0229811 … 0.0 0.0], [0.878387 1.88879 … 0.205348 0.0; 0.0 0.0 … 18.6921 1.93398; … ; 0.0 0.0 … 0.0 2.13423; 0.0 0.0 … 0.0 0.314117], LowRankModels.ConvergenceHistory("ProxGradGLRM", [Inf, 7.84535e5, 48.3097, 44.7384, 43.7999, 43.2805, 43.1153, 43.0221, 42.9711, 42.8486  …  37.472, 37.3815, 37.3368, 37.3042, 37.2725, 37.2436, 37.2171, 37.1914, 37.1675, 37.1456], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.00062108, 0.00191307, 0.00327301, 0.00387096, 0.00439882, 0.00491595, 0.00539303, 0.00587893, 0.00636888  …  0.0351872, 0.0356302, 0.0358822, 0.0363371, 0.0365932, 0.0369854, 0.0374954, 0.0380075, 0.0386164, 0.0388873], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0))

In [27]:
# did it converge? 
plot(ch.objective[2:end])
xlabel!("iteration")
ylabel!("objective")
yaxis!(:log)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 3 
 
 
 10 
 
 
 4 
 
 
 10 
 
 
 5 
 
 
 iteration 
 
 
 objective 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.1181,7.87402 60.5424,362.82 65.9668,365.632 71.3911,366.408 76.8154,366.845 82.2397,366.985 87.664,367.064 93.0884,367.107 98.5127,367.212 103.937,367.255 
 109.361,367.288 114.786,367.318 120.21,367.345 125.634,367.37 131.059,367.395 136.483,367.421 141.907,367.45 147.332,367.483 152.756,367.52 158.18,367.56 
 163.605,367.603 169.029,367.649 174.453,367.697 179.878,367.746 185.302,367.798 190.726,367.852 196.15,367.908 201.575,367.965 206.999,368.023 212.423,368.083 
 217.848,368.144 223.272,368.207 228.696,368.271 234.121,368.336 239.545,368.401 244.969,368.469 250.394,368.539 255.818,368.611 261.242,368.682 266.667,368.752 
 272.091,368.822 277.515,368.897 282.94,368.973 288.364,369.051 293.788,369.133 299.213,369.219 304.637,369.313 310.061,369.42 315.486,369.546 320.91,369.651 
 326.334,369.763 331.759,369.87 337.183,369.928 342.607,369.98 348.031,370.029 353.456,370.076 358.88,370.125 364.304,370.17 369.729,370.211 375.153,370.25 
 380.577,370.286 386.002,370.319 391.426,370.349 396.85,370.377 402.275,370.404 407.699,370.428 413.123,370.452 418.548,370.476 423.972,370.502 429.396,370.529 
 434.821,370.574 440.245,370.612 445.669,370.662 451.094,370.71 456.518,370.759 461.942,370.827 467.367,370.889 472.791,370.967 478.215,371.038 483.64,371.118 
 489.064,371.207 494.488,371.29 499.913,371.387 505.337,371.474 510.761,371.573 516.185,371.662 521.61,371.76 527.034,371.848 532.458,371.942 537.883,372.033 
 543.307,372.121 548.731,372.209 554.156,372.253 559.58,372.285 565.004,372.316 570.429,372.345 575.853,372.37 581.277,372.396 586.702,372.419 592.126,372.441 
 
 "/>
 
 
 
 
 y1

This runs an alternating directions proximal gradient method on glrm to find the $X$ and $W$ minimizing the objective function.

In [18]:
X

5×10 Array{Float64,2}:
 0.0  0.0        0.0       0.0       …  0.0        0.389987  0.0       0.0
 0.0  0.141492   0.0       5.53853      0.0963035  0.0       0.0       0.0
 0.0  0.0        0.116017  0.0          0.0        0.0       0.138903  0.0
 0.0  0.156508   0.0       2.37337      0.0        0.0       0.0       0.0
 0.0  0.0229811  0.0       0.398914     0.0        0.0       0.0       0.0

In [19]:
W

5×10 Array{Float64,2}:
 0.878387  1.88879  0.0       0.0      …  11.6352    0.205348  0.0     
 0.0       0.0      0.0       2.13951      6.29852  18.6921    1.93398 
 0.0       7.29906  0.769214  0.0          0.0       0.0       0.0     
 0.0       0.0      0.887849  0.0          1.57364   0.0       2.13423 
 0.0       0.0      0.0       0.0          3.17405   0.0       0.314117

To allow further convergence, try running for more iterations. 
You can just call fit!(glrm) again, or increase the number of iterations.

# Initialization

You'll also often improve convergence, or even converge to a better solution, with good initialization:

In [31]:
init_svd!(glrm)
X,W,ch_svd = fit!(glrm)
plot(ch_svd.objective[2:end])
xlabel!("iteration")
ylabel!("objective")
yaxis!(:log)

Fitting GLRM
Iteration 10: objective value = 19.391931654995187
Iteration 20: objective value = 18.869887034619236
Iteration 30: objective value = 18.741482514465858


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 10 
 
 
 1.50 
 
 
 10 
 
 
 1.75 
 
 
 10 
 
 
 2.00 
 
 
 iteration 
 
 
 objective 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.1181,7.87402 69.6318,266.329 84.1456,311.499 98.6593,331.251 113.173,345.393 127.687,354.499 142.2,359.334 156.714,362.501 171.228,364.956 185.742,366.658 
 200.255,367.673 214.769,368.313 229.283,368.9 243.797,369.447 258.31,369.979 272.824,370.34 287.338,370.584 301.851,370.804 316.365,370.997 330.879,371.168 
 345.393,371.325 359.906,371.479 374.42,371.63 388.934,371.768 403.448,371.9 417.961,372.028 432.475,372.156 446.989,372.262 461.502,372.279 476.016,372.297 
 490.53,372.314 505.044,372.332 519.557,372.35 534.071,372.369 548.585,372.387 563.099,372.408 577.612,372.428 592.126,372.441 
 "/>
 
 
 
 
 y1

# Treating different columns differently
The losses argument can also be an array of loss functions, with one for each column (in order). For example, for a data set with 3 columns, you could use:

In [24]:
losses = Loss[QuadLoss(), LogisticLoss(), HingeLoss()]

LoadError: UndefVarError: Loss not defined

Similiarly, the $r_w$ argument can be an array of regularizers, with one for each column (in order). For example, for a data set with 3 columns, you could use:

In [32]:
rw = Regularizer[QuadReg(1), QuadReg(10), OneReg()]

3-element Array{LowRankModels.Regularizer,1}:
 LowRankModels.QuadReg(1.0) 
 LowRankModels.QuadReg(10.0)
 LowRankModels.OneReg(1.0)  

# Example: PCA

In [33]:
# minimize ||Y - XW||^2
function fit_pca(m,n,k)
	# matrix to encode
	Y = randn(m,k)*randn(k,n)
	loss = QuadLoss()
	r = ZeroReg()
	glrm = GLRM(Y,loss,r,r,k)
	X,W,ch = fit!(glrm)
	println("Convergence history:",ch.objective)
	return Y,X,W,ch
end

fit_pca (generic function with 1 method)